<a href="https://colab.research.google.com/github/3D-Lidar-FYP/Sensor_Fusion_FYP/blob/main/Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras==2.1.5

In [ ]:
%tensorflow_version 1.x 

In [ ]:
import numpy as np
import sys
import os
import tensorflow as tf
from keras import optimizers
from keras.layers import Input, Add, concatenate
from keras.layers.merge import Concatenate
from keras.models import Model
from keras.layers import Dense, Flatten, Reshape, Dropout
from keras.layers import Convolution1D, MaxPooling1D, BatchNormalization
from keras.layers import Lambda
from keras.utils import np_utils
from keras.models import load_model
import h5py
from matplotlib.pyplot import imshow
import cv2
import glob
import math
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.imagenet_utils import decode_predictions

Using TensorFlow backend.


In [ ]:
##Adding this code myself-Ali Abdullah

# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 160983 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.24-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.24-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.24-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [ ]:

# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [ ]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

print ('Files in Drive:')
!ls drive/Colab\ Notebooks

# Create a file in Drive.
!echo "This newly created file will appear in your Drive file list." > drive/created.txt

Files in Drive:
'Copy of Welcome To Colaboratory'    train_points.npy
'Data Processing (bdb373c6).ipynb'   Untitled0.ipynb
'Data Processing.ipynb'		     Untitled10.ipynb
 Evaluation.ipynb		     Untitled1.ipynb
 intermediate_output.npy	     Untitled2.ipynb
 my_model_weights_450.h5	     Untitled3.ipynb
 processed_image.npy		     Untitled4.ipynb
 train_classes.npy		     Untitled5.ipynb
 trainHistoryDict_history450	     Untitled6.ipynb
 Training.ipynb			     Untitled8.ipynb
 train_labels.npy


PATH TO DATA:

In [ ]:
#testing the data processing
pcd_path = "drive/Data For FYP - Copy/training/velodyne/"
label_path = "drive/Data For FYP - Copy/training/labels/"
calib_path = "drive/Data For FYP - Copy/training/calib/"
img_path = "drive/Data For FYP - Copy/training/images/"

Generate Training Data for PointNet:
(Data generated, processed, and stored in 'train_points.npy' and 'train_labels.npy')

In [ ]:
#loading files and paths
BASE_DIR = os.path.dirname(os.path.abspath('__file__'))
sys.path.append(BASE_DIR)

def load_pc_from_bin(bin_path):
    """Load a bin file to a np array"""
    num_select = 2048
    selected_points =[]
    obj = np.fromfile(bin_path, dtype = np.float32).reshape(-1,4)
    pc = filter_camera_angle(obj)
    index = np.random.choice(pc.shape[0], num_select, replace=False)
    for i in range(len(index)):
        selected_points.append(pc[index[i]][0:3])
    selected_points = np.array(selected_points).reshape(1,-1,3)# return N*3 array
    return selected_points



def read_calib_file(calib_path):
    """Read a calibration file."""
    data = {}
    with open(calib_path, 'r') as f:
        for line in f.readlines():
            if not line or line == "\n":
                continue
            key, value = line.split(':', 1)
            try:
                data[key] = np.array([float(x) for x in value.split()])
            except ValueError:
                pass
    return data


def proj_to_velo(calib_data):
    """Projection matrix to 3D axis for 3D Label"""
    rect = calib_data["R0_rect"].reshape(3, 3)
    velo_to_cam = calib_data["Tr_velo_to_cam"].reshape(3, 4)
    inv_rect = np.linalg.inv(rect)
    inv_velo_to_cam = np.linalg.pinv(velo_to_cam[:, :3])
    return np.dot(inv_velo_to_cam, inv_rect)


def read_label_from_txt(label_path):
    """Read label from txt file."""
    text = np.fromfile(label_path)
    bounding_box = []
    with open(label_path, "r") as f:
        labels = f.read().split("\n")
        for label in labels:
            if not label:
                continue
            label = label.split(" ")
            if (label[0] == "DontCare"):
                continue
            y_class = [int(label[0]=="Car"), int(label[0]=="Van"), int(label[0]=="Pedestrian")]
            if label[0] == ("Car") or label[0]=="Van" or label[0]=="Pedestrian": #  or "Truck"
                y_labels = list(np.hstack((label[8:15], y_class)))
                bounding_box.append(y_labels)
                break

    if bounding_box:
        data = np.array(bounding_box, dtype=np.float32)
        return data[:, 3:6], data[:, :3], data[:, 6], data[:,7:] 
    else:
        return None, None, None, None

def read_labels(label_path, label_type, calib_path=None, is_velo_cam=False, proj_velo=None):
    """Read labels from txt file.
    Original Label value is shifted about 0.27m from object center.
    So need to revise the position of objects.
    """
    if label_type == "txt": #TODO
        places, size, rotates, y_class = read_label_from_txt(label_path)
        if places is None:
            return None, None, None, None
        rotates = np.pi / 2 - rotates
        dummy = np.zeros_like(places)
        dummy = places.copy()
        if calib_path:
            places = np.dot(dummy, proj_velo.transpose())[:, :3]
        else:
            places = dummy
        if is_velo_cam:
            places[:, 0] += 0.27 #Original Label value is shifted about 0.27m from object center

    data_combined = []
    for p, r, s , cl in zip(places, rotates, size, y_class):
        ps = np.hstack((cl[:],p[:],s[:]))
        data_combined.append(list(np.append(ps, r)))
        
    return places, rotates, size, y_class

def get_boxcorners(places, rotates, size):
    """Create 8 corners of bounding box from bottom center."""
    corners = []
    for place, rotate, sz in zip(places, rotates, size):
        x, y, z = place
        h, w, l = sz
        if l > 10:
            continue

        corner = np.array([
            [x - l / 2., y - w / 2., z],
            [x + l / 2., y - w / 2., z],
            [x - l / 2., y + w / 2., z],
            [x - l / 2., y - w / 2., z + h],
            [x - l / 2., y + w / 2., z + h],
            [x + l / 2., y + w / 2., z],
            [x + l / 2., y - w / 2., z + h],
            [x + l / 2., y + w / 2., z + h],
        ])

        corner -= np.array([x, y, z])

        rotate_matrix = np.array([
            [np.cos(rotate), -np.sin(rotate), 0],
            [np.sin(rotate), np.cos(rotate), 0],
            [0, 0, 1]
        ])

        a = np.dot(corner, rotate_matrix.transpose())
        a += np.array([x, y, z])
        corners.append(a)
    return np.array(corners)

def filter_camera_angle(places):
    """Filter camera angles for KiTTI Datasets"""
    bool_in = np.logical_and((places[:, 1] < places[:, 0] - 0.27), (-places[:, 1] < places[:, 0] - 0.27)) #Original Label value is shifted about 0.27m from object center
    # bool_in = np.logical_and((places[:, 1] < places[:, 0]), (-places[:, 1] < places[:, 0]))
    return places[bool_in]

def center_to_sphere(places, size, resolution=0.50, min_value=np.array([0., -50., -4.5]), scale=4, x=(0, 90), y=(-50, 50), z=(-4.5, 5.5)):
    """Convert object label to Training label for objectness loss"""
    x_logical = np.logical_and((places[:, 0] < x[1]), (places[:, 0] >= x[0]))
    y_logical = np.logical_and((places[:, 1] < y[1]), (places[:, 1] >= y[0]))
    z_logical = np.logical_and((places[:, 2] < z[1]), (places[:, 2] >= z[0]))
    xyz_logical = np.logical_and(x_logical, np.logical_and(y_logical, z_logical))
    center = places.copy()
    center[:, 2] = center[:, 2] + size[:, 0] / 2.
    sphere_center = ((center[xyz_logical] - min_value) / (resolution * scale)).astype(np.int32)
    return sphere_center

def sphere_to_center(p_sphere, resolution=0.5, scale=4, min_value=np.array([0., -50., -4.5])):
    """from sphere center to label center"""
    center = p_sphere * (resolution*scale) + min_value
    return center

#derived from yukitsuji/3D_CNN_tensorflow
def process(velodyne_path, label_path=None, calib_path=None, dataformat="bin", label_type="txt", is_velo_cam=False):
    p = []
    pc = None
    bounding_boxes = None
    places = None
    rotates = None
    size = None
    proj_velo = None
    
    filenames_velo = [d for d in sorted(os.listdir(velodyne_path)) if d[0]!='.']
    train_points = None
    train_labels = None
    train_classes = None
    for d in filenames_velo:
        value = d[0:6]
        print(value)
        velo_path = velodyne_path + value + '.bin'
        cal_path = calib_path + value + '.txt'
        lab_path = label_path + value + '.txt'
        cur_points = load_pc_from_bin(velo_path)
        cur_calib = read_calib_file(cal_path)
        proj_velo = proj_to_velo(cur_calib)[:, :3]
        places, rotates, size, classes= read_labels(lab_path, label_type, cal_path , is_velo_cam=is_velo_cam, proj_velo=proj_velo)
        corners = get_boxcorners(places, rotates, size)
        if places is None:
            continue
        if train_points is None:
            train_labels = corners
            train_points = cur_points
            train_classes = classes
        else:
            train_points = np.concatenate((train_points, cur_points), axis = 0)
            train_labels = np.concatenate((train_labels, corners), axis = 0)
            train_classes = np.concatenate((train_classes, classes), axis =0)
            
    return train_points, train_labels, train_classes

In [ ]:
train_points, train_labels, train_classes = process(pcd_path, label_path, calib_path=calib_path, dataformat="bin", is_velo_cam=True)

In [ ]:
print(train_classes.shape)
np.save('drive/Colab Notebooks/train_classes.npy', train_classes)
np.save('drive/Colab Notebooks/train_points.npy', train_points)
np.save('drive/Colab Notebooks/train_labels.npy', train_labels)

(5401, 3)


In [ ]:
lab = np.load('drive/Colab Notebooks/train_labels.npy')
print(lab.shape)
#print(train_labels.reshape((7481,24)).shape)
#print (train_labels.reshape(5401,24).shape)


(5401, 8, 3)


Generate image data and processing using Resnet-50 model:
(Data generated, processed, and activation layer is stored in 'intermediate_output.npy')

In [ ]:
#run this cell only if images need to be processed again
filenames_imgs = [img_path+d for d in sorted(os.listdir(img_path)) if d[0]!='.']
batch_images = None
for d in filenames_imgs:
    print(d)
    original = load_img(d, target_size=(224, 224))
    numpy_image = img_to_array(original)
    image_batch_cur = np.expand_dims(numpy_image, axis=0)
    if batch_images is None:
        batch_images = image_batch_cur
    else:
        batch_images = np.concatenate((batch_images,image_batch_cur ), axis = 0)



Streaming output truncated to the last 5000 lines.
drive/Data For FYP - Copy/training/images/000402.png
drive/Data For FYP - Copy/training/images/000403.png
drive/Data For FYP - Copy/training/images/000404.png
drive/Data For FYP - Copy/training/images/000405.png
drive/Data For FYP - Copy/training/images/000406.png
drive/Data For FYP - Copy/training/images/000407.png
drive/Data For FYP - Copy/training/images/000408.png
drive/Data For FYP - Copy/training/images/000409.png
drive/Data For FYP - Copy/training/images/000410.png
drive/Data For FYP - Copy/training/images/000411.png
drive/Data For FYP - Copy/training/images/000412.png
drive/Data For FYP - Copy/training/images/000413.png
drive/Data For FYP - Copy/training/images/000414.png
drive/Data For FYP - Copy/training/images/000415.png
drive/Data For FYP - Copy/training/images/000416.png
drive/Data For FYP - Copy/training/images/000417.png
drive/Data For FYP - Copy/training/images/000418.png
drive/Data For FYP - Copy/training/images/000419

In [ ]:
from keras.applications import resnet50
resnet_model = resnet50.ResNet50(weights='imagenet')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

102858752/102853048 [==============================] - 5s 0us/step


In [ ]:
#run this cell only if images need to be processed again. Otherwise, use the saved file in next cell
processed_image = resnet50.preprocess_input(batch_images.copy())
np.save('drive/Colab Notebooks/processed_image.npy', processed_image)

In [ ]:
resnet_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [ ]:
#run this cell only if images need to be processed again by resnet. Otherwise, use the saved file in next cell
processed_images = np.load('drive/Colab Notebooks/processed_image.npy')

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

#extract final layer called fc1000 with size 1 by 1000 
layer_name = 'fc1000'
layer = resnet_model.get_layer(layer_name)

intermediate_layer_model = Model(inputs=resnet_model.input,
                                 outputs=layer.output)
intermediate_layer_model.summary()
intermediate_output = intermediate_layer_model.predict(processed_images)
print("Shape of Processed Images: ", processed_images.shape)
print("Layer used: ", layer.name)
print("Shape of layer output:", intermediate_output.shape)

np.save('drive/Colab Notebooks/intermediate_output.npy', intermediate_output)


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [ ]:
intermediate_output = np.load('drive/Colab Notebooks/intermediate_output.npy')
print(intermediate_output.shape)